## Traditional machine learning
Feature(s): 'SMILES' column

- convert to morgan fingerprint

Target: E_coh_bin,T_g_bin,R_gyr_bin,Densities_bin

- 4 targets
- each a binary class of a monomer property

In [1]:
import sys
import os

# Append the parent directory of your package to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..')))

In [2]:
import pandas as pd
import numpy as np
import zipfile

path_to_dataset = 'train_ben.csv'
csv_filename = 'train_ben.csv'

# Open the file, Correct the encoding and sep if necessary
if path_to_dataset.endswith('.zip'):
    with zipfile.ZipFile(path_to_dataset, 'r') as z:
        # Open the CSV file within the ZIP file
        with z.open(csv_filename) as f:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(f, sep=',', on_bad_lines='warn')
else:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(path_to_dataset, sep=',', on_bad_lines='warn')


print('Count of unique smiles:', df.SMILES.unique().shape[0])
print('Count of all of the smiles:', df.shape[0])

Count of unique smiles: 410
Count of all of the smiles: 410


In [3]:
from MLPipeline import MLmodel, BinTheTarget

Target = ['E_coh_bin','T_g_bin','R_gyr_bin','Densities_bin']
Features = ['SMILES']
Feature_types = ['SMILES']
input = df

/home/lsmo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = MLmodel(modelType='RandomForestClassifier',
                    df=input,
                    target=Target,
                    features=Features,
                    feature_types=Feature_types)

# get the values (input and output) of the model
X_train, X_test, y_train, y_test = model.getValues()

2024-09-12 17:34:10.177 | INFO     | MLPipeline:__post_init__:134 - ndim y_train: 2
2024-09-12 17:34:10.178 | INFO     | MLPipeline:__post_init__:135 - ndim x_train: 2
2024-09-12 17:34:10.178 | INFO     | MLPipeline:__post_init__:136 - shape y_train: (50, 4)
2024-09-12 17:34:10.179 | INFO     | MLPipeline:__post_init__:137 - shape x_train: (50, 512)


In [5]:
model.train()
model.evaluate()

RandomForestClassifier model trained successfully.
Accuracies for each target in RandomForestClassifier: [0.7, 0.68, 0.82, 0.72]


[0.7, 0.68, 0.82, 0.72]

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.base import clone

def objective(trial, model_instance):
    """
    Objective function for Optuna to minimize.
    """
    # Define hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_categorical('max_depth', [None, 10, 20, 30, 40]),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 6),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
    }


    # Clone the model to ensure a fresh instance each trial
    model_to_clone = model_instance.model.estimator
    model_clone = clone(model_to_clone)
    model_clone.set_params(**params)
    
    # Define the score metric
    scoring = 'accuracy'

    # Perform cross-validation
    scores = cross_val_score(model_clone, model_instance.X_train, model_instance.y_train, cv=model_instance.cv, scoring=scoring)

    # Return the average score across all folds
    return scores.mean()

In [7]:
model = MLmodel(modelType='RandomForestClassifier', df=input, target=Target, 
                features=['SMILES'], hyperparameter_tuning=True,
                feature_types=Feature_types,
                optimization_method='optuna', objective=lambda trial: objective(trial, model))

model.train()
predictions = model.predict()
model.evaluate()

2024-09-12 17:34:11.020 | INFO     | MLPipeline:__post_init__:134 - ndim y_train: 2
2024-09-12 17:34:11.020 | INFO     | MLPipeline:__post_init__:135 - ndim x_train: 2
2024-09-12 17:34:11.021 | INFO     | MLPipeline:__post_init__:136 - shape y_train: (50, 4)
2024-09-12 17:34:11.021 | INFO     | MLPipeline:__post_init__:137 - shape x_train: (50, 512)
[I 2024-09-12 17:34:11,022] A new study created in memory with name: no-name-92d337e9-34cf-468c-870a-884ea6cde08b
[I 2024-09-12 17:34:11,620] Trial 0 finished with value: 0.06000000000000001 and parameters: {'n_estimators': 149, 'max_depth': 40, 'min_samples_split': 11, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.06000000000000001.
[I 2024-09-12 17:34:12,333] Trial 1 finished with value: 0.06000000000000001 and parameters: {'n_estimators': 181, 'max_depth': None, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 0.0600

Best RandomForestClassifier model trained successfully with hyperparameter tuning using Optuna.
Best hyperparameters: {'n_estimators': 119, 'max_depth': 40, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': True}
RandomForestClassifier model trained successfully.
Accuracies for each target in RandomForestClassifier: [0.58, 0.6, 0.78, 0.54]


[0.58, 0.6, 0.78, 0.54]